Once the data is uploaded to EIS-Geospatial Component (PAIRS), you can use this notebook to expose a query as custom layer.   

# Module Imports

In [1]:
import requests 
import json
import pandas as pd
import numpy as np
import time

In [2]:
import jwt

In [40]:
API_KEY = ""

In [41]:
import requests

url = "https://auth-b2b-twc.ibm.com/Auth/GetBearerForClient"

payload={"apiKey":API_KEY,
         "clientId":"ibm-agro-api"}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [5]:
response.status_code

200

In [6]:
if response:
    AUTH_TOKEN = response.json()["access_token"]

In [42]:
def check_jwt(jwt_token):

    response = jwt.decode(jwt_token, options={"verify_signature": False})
    org = str(response['account_name'])
    iss = str(response['iss'])
    
    print(org,iss)


In [9]:
check_jwt(jwt_token=AUTH_TOKEN)

wocTrialWorldFuel https://auth-b2b-twc.ibm.com


# Register Custom layer

To register a custom layer, one need to decide on the spatial and temporal extent of a PAIRS layer and come put with a valid PAIRS raster query. One can experiment with different spatial and temporal extents using the PAIRS data expolorer at https://ibmpairs.mybluemix.net/data-explorer

In case of AOI (Area of Interest) polygons, PAIRS has pre-defined set of AOIs with ids assigned to each aoi. For Ecample AOI id 125 corresponds to the US-State of Oregon. 

One can get a full list of AOIs by using the API call : https://pairs.res.ibm.com/ws/queryaois/repository/search?name=SEARCH_TERM


In this example we will use the Fire Danger Rating Layer (id: 50039) for th State of Oregon. 

In [10]:
fire_pairs_payload = {
    "spatial": {
        "type": "poly",
        "aoi": "126"
    },
    "temporal": {
        "intervals": [
            {
                "start": "2021-08-13",
                "end": "2021-08-15"
            }
        ]
    },
    "layers": [
        {
            "id": "50039",
            "type": "raster"
        }
    ]
}

Pick a name for `analyticsName` that describes the cutom layer. This name will be displayed on th EIS UI.

In [11]:
registration_payload = {
    "pairsPayload":  json.dumps(fire_pairs_payload)   ,
    "analyticsName": "Fire Danger Rating Colorado August 2021"
}

To register a PAIRS payload as a custom layer, we make a POST call to https://foundation.agtech.ibm.com/v2/layer/analytics/metadata and the response containts a `analyticsUuid` which is a unique identifier for the custom layer. We will use this UUID to add a UI Block in the next step.

In [12]:
url = "https://foundation.agtech.ibm.com/v2/layer/analytics/metadata"

headers = {
  'Authorization': 'Bearer {}'.format(AUTH_TOKEN),
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=json.dumps(registration_payload))

print(json.dumps(response.json(),indent=2))


[
  {
    "analyticsUuid": "8468a886-e6ad-4ec2-aa65-eb05864bc3c6",
    "layerId": "50039",
    "baseComputationId": "1631678400_40024186"
  }
]


In [13]:
response.text

'[{"analyticsUuid":"8468a886-e6ad-4ec2-aa65-eb05864bc3c6","layerId":"50039","baseComputationId":"1631678400_40024186"}]'

In [14]:
layer_info = response.json()
#print(layer_info)

analytics_uuid = layer_info[0]["analyticsUuid"]
pairs_layer_id = layer_info[0]["layerId"]
base_computation_id = layer_info[0]["baseComputationId"]

# Customizing Colorsheme for the custom-layers on the UI

This section has the

In [15]:
def hex_to_rgb(hex_code):
    h = hex_code.lstrip('#')
    return list(int(h[i:i+2], 16) for i in (0, 2, 4))

In [16]:
pairs_url = "https://pairs.res.ibm.com/v2/datalayers/{}".format(pairs_layer_id)

In [17]:
headers = {
  'Authorization': 'Bearer {}'.format(AUTH_TOKEN)
}

In [18]:
response = requests.request("GET", pairs_url, headers=headers)

In [19]:
response.json()

{'id': '50039',
 'created_at': 1593726629784,
 'updated_at': 1631639727738,
 'data_source_links': [],
 'description_short': 'A Fire Danger Rating level taking into account current and antecedent weather, fuel types, and both live and dead fuel moisture.',
 'description_long': 'The United States Forest Service (USFS) - Wildland Fire Assessment System (WFAS) collect observations from the field along with weather observations to compute a Fire Danger Rating. The Fire Danger Rating level takes into account current and antecedent weather, fuel types, and both live and dead fuel moisture.  The data in this layer is derived from the daily WFAS reports. The categorical Fire Danger Ratings low/medium/high/very high/extreme are converted to floating point numbers 0.2/0.4/0.6/0.8/1.0. Rasterization of the data uses inverse distance weighted interpolation.',
 'description_links': [],
 'latitude_min': -90.0,
 'latitude_max': 90.0,
 'longitude_min': -180.0,
 'longitude_max': 180.0,
 'temporal_min': 

In [20]:
color_block = response.json()["colorTable"]

In [21]:
color_block

{'id': 58, 'name': 'Red', 'colors': 'FFFFB2,FECC5C,FD8D3C,F03B20,BD0026'}

In [22]:
for color in color_block["colors"].split(','):
    print(color)
    print(hex_to_rgb(color))

FFFFB2
[255, 255, 178]
FECC5C
[254, 204, 92]
FD8D3C
[253, 141, 60]
F03B20
[240, 59, 32]
BD0026
[189, 0, 38]


## PAIRS layer metadata

We Need to wait till the PAIRS job is finished before going to the next step. The following block keeps requesting the job status till the job is done. 

In [23]:
base_computation_id

'1631678400_40024186'

In [38]:
pairs_jobs_status_url = f"https://pairs.res.ibm.com/v2/queryjobs/{base_computation_id}"

payload={}
headers = {
  'Authorization': f'Bearer {AUTH_TOKEN}'
}

job_finished = False
while not job_finished:
    job_status_response = requests.request("GET", pairs_jobs_status_url, headers=headers, data=payload)
    if job_status_response.status_code == 200:
        job_status = job_status_response.json()["status"]
        if (job_status == "Running" ) or (job_status == "Writing"):
            print(f"Waiting for PAIRS job to finish..current status:{job_status}")
            print("waiting for 15 sec..")
            time.sleep(15)
        elif (job_status == "Succeeded" ):
            print("job Done!!")
            job_finished = True
        elif (job_status == "NoDataFound" ):
            print("PAIRS found no data in the selected geometry and date range..!!")
            print("Please select a different date range..")
    else:
        job_status_response.raise_for_status()


job Done!!


In [25]:
job_status_response = requests.request("GET", pairs_jobs_status_url, headers=headers, data=payload)

In [26]:
job_status_response.text

'{\n  "id" : "1631678400_40024186",\n  "status" : "Succeeded",\n  "start" : 1631718424186,\n  "folder" : "1631718424170",\n  "swLat" : 36.939194,\n  "swLon" : -109.09254,\n  "neLat" : 41.04213,\n  "neLon" : -101.99445,\n  "exPercent" : 0.0,\n  "flag" : true,\n  "hadoopId" : "1602345287025_5087641",\n  "ready" : true,\n  "rtStatus" : "Succeeded",\n  "statusCode" : 20\n}'

In [27]:
pairs_metadata_url = "https://pairs.res.ibm.com/v2/queryjobs/{}/layers".format(base_computation_id)

headers = {
  'Authorization': 'Bearer {}'.format(AUTH_TOKEN)
}

metadata_response = []    

metadata_response = requests.request("GET", pairs_metadata_url, headers=headers)




In [28]:
metadata_response.json()

[{'name': '1631678400_40024186WildlandfireUSFS-Firedangerrating-08_14_2021T000000',
  'datasetId': 299,
  'dataset': 'Wildland fire (USFS)',
  'datalayerId': '50039',
  'datalayer': 'Fire danger rating',
  'timestamp': 1628899200000,
  'min': 0.2,
  'max': 0.697,
  'colorTableId': '58',
  'unitsbl': '[0-1]',
  'type': 'raster',
  'geoserverUrl': 'https://pairs.res.ibm.com:8080/geoserver04',
  'geoserverWS': 'pairs'},
 {'name': '1631678400_40024186WildlandfireUSFS-Firedangerrating-08_15_2021T000000',
  'datasetId': 299,
  'dataset': 'Wildland fire (USFS)',
  'datalayerId': '50039',
  'datalayer': 'Fire danger rating',
  'timestamp': 1628985600000,
  'min': 0.4,
  'max': 0.669,
  'colorTableId': '58',
  'unitsbl': '[0-1]',
  'type': 'raster',
  'geoserverUrl': 'https://pairs.res.ibm.com:8080/geoserver04',
  'geoserverWS': 'pairs'}]

In [29]:
layer_min = 0
layer_max = 1

In [30]:
def generate_color_steps(color_block,layer_min,layer_max):
    
    hex_colors = color_block["colors"].split(',')
    value_range = np.linspace(layer_min, layer_max, num=len(hex_colors))
    
    color_steps = []
    for idx,val in enumerate(hex_colors):
        
        rgba_list = hex_to_rgb(val)
        rgba_list.append(255)
        dct = {"step": value_range[idx], "rgba":rgba_list}
        
        color_steps.append(dct)
    
    return color_steps
        
    
    

In [31]:
color_steps = generate_color_steps(color_block,layer_min,layer_max)

print(color_steps)

[{'step': 0.0, 'rgba': [255, 255, 178, 255]}, {'step': 0.25, 'rgba': [254, 204, 92, 255]}, {'step': 0.5, 'rgba': [253, 141, 60, 255]}, {'step': 0.75, 'rgba': [240, 59, 32, 255]}, {'step': 1.0, 'rgba': [189, 0, 38, 255]}]


# Adding UI block

In [32]:
url = "https://api.wsitrader.com/api/v1/IMAP/put-layer-config-block"

In [33]:
registration_payload["analyticsName"]

'Fire Danger Rating Colorado August 2021'

In [34]:
ui_block_payload = {
    "VIEWERSHIP_ROLE": "ALL",
    "CONFIG_BLOCK": {
        "id": registration_payload["analyticsName"],
        "modelRegistryId": None,
        "displayName": registration_payload["analyticsName"],
        "provider": None,
        "layerType": "grid",
        "isSelected": False,
        "isActive": False,
        "enableValidity": False,
        "lastUpdatedUtc": None,
        "coverageArea": "Custom",
        "dataAttributes": {
            "url": "https://foundation.agtech.ibm.com/v2",
            "uuid": analytics_uuid
        },
        "menuIconUrl": None,
        "legendUrl": "",
        "styleProperties": {
            "palette": {
                "COLOR_STEPS": color_steps
       
            },
            "unit": "",
            "isInterpolated": True,
            "extendMinimumColor": False,
            "extendMaximumColor": True,
            "invalidDataValue": -9999
        }
    }
}

In [35]:
headers = {
  'Authorization': 'Bearer {}'.format(AUTH_TOKEN),
  'Content-Type': 'application/json'
}

In [36]:
response = requests.put(url, headers= headers, data=json.dumps(ui_block_payload))

In [37]:
response.text

'Block added'

# Deleting UI block

In [82]:
block_id = registration_payload["analyticsName"]
block_id

'Fire Danger Rating naperville-elgin-16980 August 2021'

In [83]:
url = "https://api.wsitrader.com/api/v1/IMAP/get-layer-config-block/{}".format(block_id)

In [84]:
headers = {
  'Authorization': 'Bearer {}'.format(jwt)
}

In [85]:
response = requests.get(url, headers= headers)

In [86]:
response.text

''

In [92]:
delete_url = "https://api.wsitrader.com/api/v1/IMAP/delete-layer-config-block/{}".format(block_id)

In [93]:
headers = {
  'Authorization': 'Bearer {}'.format(jwt)
}

In [94]:
response = requests.delete(delete_url, headers= headers)

In [95]:
response.text

'Block removed'